Perform the following tasks here 

1.   bind LangChain to the HuggingFace Embeddings,
2.   feed the pipeline with Similarity Search into a brand new created vectorized database with our documents
3.   give it to LLM with the HuggingFace Access Token together with our question
4.   get the answer 



Langchain here acts as the framework to connect multiple components like huggingFace's LLM (Model) with vectorDb like chromadb and similarity search by FAISS 

In [2]:
%%capture
!pip install langchain
!pip install huggingface_hub
!pip install sentence_transformers
!pip install faiss-cpu
!pip install unstructured
!pip install chromadb
!pip install Cython
!pip install tiktoken
!pip install unstructured[local-inference]

Login to the huggingface hub using the access token in the Hugging Face account settings 

In [3]:
import os
import requests

os.environ["HUGGINGFACEHUB_API_TOKEN"] = "<INSERT_YOUR_TOKEN_HERE>"

Two parts in this notebook: 


1.   Generate QnA using raw text given as context
2.   Generate QnA using pdf (will update later in another project)

But below code is showing that langchain even provides the pdf loader in the document_loaders module



In [4]:
from langchain.document_loaders import TextLoader  #for textfiles
from langchain.text_splitter import CharacterTextSplitter #text splitter
from langchain.embeddings import HuggingFaceEmbeddings #for using HugginFace models
# Vectorstore: https://python.langchain.com/en/latest/modules/indexes/vectorstores.html
from langchain.vectorstores import FAISS  #facebook vectorizationfrom
from langchain.chains.question_answering import load_qa_chain

from langchain import HuggingFaceHub

from langchain.document_loaders import UnstructuredPDFLoader  #load pdf
from langchain.indexes import VectorstoreIndexCreator #vectorize db index with chromadb
from langchain.chains import RetrievalQA
from langchain.document_loaders import UnstructuredURLLoader  #load urls into docoument-loader


Talk to TXT files

In [20]:
trial_text = f"""
What is the Science of Living ?\
It is important to understand what is life before understanding what is the science of living.\
Life means the Live Internal Force of Eternity. Eternity means soul, sent by God. Body is not life but just a living or dead body. \
Life is self, i.e. the internal force of eternity is self. So it is self that constitutes life and not the body. \
Every birth we will get a new body but life, i.e. soul will remain as it is throughout all the births. Hence it should be clear what is living body, \
dead body and life. Life is because of self and not because of body. In Nutshell, life is the living internal force of eternity.\
In fetal life, the foetus becomes independently viable only after the self is embodied. This happens around the third to fourth month.\
If the self is not embodied, the foetus will die. Hence foetus life depends on the self. Until the fourth month, there is no identity of a foetus. \
It is just a scientific body. Hence now it is clear that self is the supreme eternal life force. \
Living body contains a self and an organic body. Self is made up of mind, intellect and soul. Whereas the organic body is made up of various systems and \
organs like the brain, digestive system, liver, respiratory system etc.
As long as the living force of eternity stays inside the body, it is a living body otherwise it is a dead body. For eg, whenever a person dies, \
people say may his soul rest in peace but they never say may his body rest in peace. As soon as a person dies, the soul takes the next body, \
whether man or animal, immediately. \
Ironically, people realise other’s souls immediately when that person dies and pray to God to keep the dead man’s soul in peace. But what should really \
happen is people should realise this fact during their life. People should realise that the soul is eternal and the body is temporary and hence perform \
the right karmas for the soul and not just for the body.
After understanding life and body, it should be easier to understand what is the science of living. Science of living, as defined by God, is the living \
with spiritual consciousness in eternal nature with continuous entireness. Eternal nature is related to supersoul. Continuous entireness means continuity \
or austerity. Austerity means priority with emotions like eating food on time. Continuity is important in every aspect of life. Our guru Dr. Om Prakash \
Mittal lives his life with austerity. For eg., he puts divine messages and divine poems every day irrespective of any situation or any location.\
Science of living is living with not only the requirements of the physical body. But it is living with both the components of the human entity. \
Human entity consists of two parts. Physical gross body and subtle self. Every living person is a human entity. We are a human entity. \
Name any person whom you know or aware of, are human entities. In the science of living, we work for both the components. However, it is a tragedy that \
people just work for their physical gross body. People do not have even five minutes for themselves. \
Hence the science of living is living with love and faith. Science of living is living with knowledge and wisdom. Science of living is living with the \
namely, punctuality, patience, persistence, perspiration and perseverance.
Science of living is living with the spiritual truths of life. There are no false beliefs in the science of living. People are going to various religious \
organisations to find God and perform fake and hollow rituals. But the science of living is living with the spiritual truths of life unlike fake religious \
beliefs. Science of living is living with self realisation. We have realised the body as we have grown up. Our feeling or desire to study, progress in \
our career, buy a house, buy furniture, buy food and all other fullfilments are done because we have realised our body. However, self or soul realisation \
is missing throughout this journey. Hence the science of living is living with self realisation. Once self realisation is done, there is no looking back. \
One needs to activate his inner sim. Just like a mobile is useless without a sim card, this body even with a soul is useless until the activation of \
inner sim or self realisation. When you will activate your inner sim, you will be able to connect your soul to the supersoul, i.e. God. \
This is called meditation which is nothing but connecting soul to supersoul. Many times people say they have been doing meditation for a few minutes or \
even hours. However meditation without self realisation is not possible.  Meditation is the last stage after self realisation. It is difficult or nearly \
impossible to do meditation without self realisation. First it is important to understand how to activate the inner sim and then think of doing meditation.\
What people call meditation is nothing but silence for a few minutes. It is important to understand that the science of living is the living where one \
can enjoy everything material provided one performs good karmas without attachment to them or anything. Suppose you take an expensive luxurious car today. \
But you should not keep attachment to that car. If for some reason you are not able to buy such a car but a cheaper or economical car and you are not \
feeling happy or not enjoying the experience of buy the cheaper car, this is pure attachment. This is not right in the science of living. Hence if you \
can afford a luxurious car, you have every right to buy such a car and enjoy life. This is all after the given fact that you do not have attachment to any\
material aspects. Hence one should evaluate if there is no attachment building up inside gradually. To cite a few examples, drinking alcohol can be good \
karma or bad karma depending upon how much you drink. But smoking is bad karma always. Any karma could be good or bad depending upon the situation.
Any karma performed that does not have bad effect or impact on yourself and the positive surroundings, it is called good karma. Performing good karmas \
gives a by product in the form of happiness. 
Science of living is living with happiness in all given situations. Be it corona like pandemic, it is still \
possible to live with happiness in the science of living. Happiness is not conditional but one can be happy in all situations in the science of living.
Science of living is living with happiness and bliss as the way of life. When happiness and bliss becomes the way of life, that means you are living with \
the science of living. Science of living is nothing but the science of happy living as happiness is the core issue in science of living.
Spirituality or science of living is the total science of gross body and subtle self. 
Spiritual living is the absolute science of living. Spirituality is the science of values. Spirituality is the science of virtues. Spirituality is the science of living this beautiful human life with happiness and bliss.
Science of living also includes the art of living. Art of living is just about the body but science of living is living with self too. Hence the science \
of living absolutely in happiness and bliss.
Before understanding the science of living, it is absolutely necessary to understand the creator of all creations, GOD. Because God is the creator of \
all creations, living or nonliving. God is also the creator of body and self. it is absolutely necessary to understand about this evolution and \
functioning of the body and self.
"""


I am using the content from science of living available to read more at www.facebook.com/scienceofhappyliving

In [21]:
with open("science_of_living.txt", "w") as f:
  f.write(trial_text)

One can even download the text from a web page like below

In [ ]:
import requests
url1 = "<SOME_VALID_WEB_LINK>"
res = requests.get(url1)
with open("downloaded_content.txt", "w") as f:
  f.write(res.text)

In [22]:
fname = "./science_of_living.txt"

Split the documents in chunks as
LLM cannot accept long instructions.

In [32]:

# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [33]:
# Document Loader
# from langchain.document_loaders import TextLoader
# loader = TextLoader(fname)
# documents = loader.load()

with open(fname) as f:
    doc = f.read()

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 400,
    chunk_overlap  = 20,
    length_function = len,
)
texts = text_splitter.create_documents([doc])

In [34]:
len(texts)

23

In [35]:
# import textwrap
# def wrap_text_preserve_newlines(text, width=110):
#     # Split the input text into lines based on newline characters
#     lines = text.split('\n')
#     # Wrap each line individually
#     wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
#     # Join the wrapped lines back together using newline characters
#     wrapped_text = '\n'.join(wrapped_lines)
#     return wrapped_text

In [36]:

# text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=10)
# docs = text_splitter.split_documents(documents)

In [37]:
texts[0].page_content

'What is the Science of Living ?It is important to understand what is life before understanding what is the science of living.Life means the Live Internal Force of Eternity. Eternity means soul, sent by God. Body is not life but just a living or dead body. Life is self, i.e. the internal force of eternity is self. So it is self that constitutes life and not the body. Every birth we will get a new'

In [38]:
len(texts[0].page_content)

398

Embeddings 

In [39]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

Use the facebook similarity search to create the vector of embeddings for the set of chunked documents

In [40]:
#Create the vectorized db
# Vectorstore: https://python.langchain.com/en/latest/modules/indexes/vectorstores.html
from langchain.vectorstores import FAISS
db = FAISS.from_documents(texts, embeddings)

One can query and find the relevant document matching this query

In [41]:
query = "What is science of living?" 

docs = db.similarity_search(query)

We got 4 sub-documents matching the above query

In [42]:
len(docs)

4

In [47]:
for d in docs:
  print()
  print(d.page_content)


After understanding life and body, it should be easier to understand what is the science of living. Science of living, as defined by God, is the living with spiritual consciousness in eternal nature with continuous entireness. Eternal nature is related to supersoul. Continuous entireness means continuity or austerity. Austerity means priority with emotions like eating food on time. Continuity is

the science of living is living with love and faith. Science of living is living with knowledge and wisdom. Science of living is living with the namely, punctuality, patience, persistence, perspiration and perseverance.

What is the Science of Living ?It is important to understand what is life before understanding what is the science of living.Life means the Live Internal Force of Eternity. Eternity means soul, sent by God. Body is not life but just a living or dead body. Life is self, i.e. the internal force of eternity is self. So it is self that constitutes life and not the body. Every bir

Use Langchain Model to query and get a human readable response from the LLM using the relevant documents

In [48]:
from langchain.chains.question_answering import load_qa_chain
from langchain import HuggingFaceHub

Note that most of the models because of large size takes time to download and may not always run completely unless downloaded locally as there's server time out error from the hugging face hub

Test with google/flan-t5-xl

In [49]:
llm=HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":0, "max_length":512})
chain = load_qa_chain(llm, chain_type="stuff")

In [50]:
chain.run(input_documents=docs, question=query)

ValueError: ignored

Test with declare-lab/flan-sharegpt-xl

In [ ]:
llm2=HuggingFaceHub(repo_id="declare-lab/flan-sharegpt-xl", model_kwargs={"temperature":0, "max_length":512})
chain = load_qa_chain(llm2, chain_type="stuff")

In [ ]:
chain.run(input_documents=docs, question=query)

⭐⭐ Test with declare-lab/flan-alpaca-large


In [ ]:
llm3=HuggingFaceHub(repo_id="declare-lab/flan-alpaca-large", model_kwargs={"temperature":0, "max_length":512})
chain = load_qa_chain(llm3, chain_type="stuff")

In [ ]:
chain.run(input_documents=docs, question=query)

In [ ]:
llm1=HuggingFaceHub(repo_id="eachadea/legacy-ggml-vicuna-13b-4bit", model_kwargs={"temperature":0, "max_length":512})
chain = load_qa_chain(llm1, chain_type="stuff")


In [ ]:
chain.run(input_documents=docs, question=query)

For testing purpose and descent results, We are trying Lamini LLM

In [51]:
llm6=HuggingFaceHub(repo_id="MBZUAI/LaMini-Flan-T5-783M", model_kwargs={"temperature":0, "max_length":512})
chain = load_qa_chain(llm6, chain_type="stuff")

In [52]:
chain.run(input_documents=docs, question=query)

'Science of living is the living with spiritual consciousness in eternal nature with continuous entireness, which includes continuity, austerity, love, faith, knowledge, wisdom, punctuality, patience, persistence, perspiration, and perseverance.'

From query to the LLM, these are the steps performed togeher

In [ ]:
query = "what is sciece of living?"
docs = db.similarity_search(query)
llm=HuggingFaceHub(repo_id="MBZUAI/LaMini-Flan-T5-783M", model_kwargs={"temperature":0, "max_length":512})

chain = load_qa_chain(llm6, chain_type="stuff")
chain.run(input_documents=docs, question=query)